In [1]:
#import cudf
#from dask_cuda import LocalCUDACluster
import distributed
#import dask_cudf
import dask
import dask.dataframe as dd
import os

In [2]:
import sys
sys.executable

'/home/ericdill/miniconda/envs/dask/bin/python'

In [3]:
unpartitioned_dir = '/mnt/storage/anaconda-parsed-logs'
output_dir = '/mnt/storage/anaconda-parsed-logs-partitioned'

In [4]:
parquet_files = []
for path, dirs, files in os.walk(unpartitioned_dir):
    for f in files:
        if f.endswith('.parquet'):
            _, base = path.split(unpartitioned_dir)
            input_file = os.path.join(path, f)
            output_file = f'{output_dir}/{base}/{f}'
            parquet_files.append((input_file, output_file))
            #print(f'{input_file} -> {output_file}')
            #print(f'reading from {input_file}. writing to {output_file}')
            #partition_file(input_file, output_file)
            #delayed_funcs.append(dask.delayed(partition_file)(input_file, output_file))
    

In [7]:
input_file, output_file = parquet_files[0]

In [8]:
ddf = dd.read_parquet(input_file)

In [12]:
ddf = ddf.reset_index()

In [21]:
ddf['day_name'] = ddf.timestamp.dt.day_name()

ddf['hour'] = ddf.timestamp.dt.hour

In [14]:
ddf.timestamp

AttributeError: 'Series' object has no attribute 'dayofweek'

In [13]:
ddf.head(1).T

,0
timestamp,2016-10-28 17:43:53.255000
bytes,3326580
city,Ashburn
continent,North America
country,United States
domain_name,compute-1.amazonaws.com
full_user_agent,conda/4.2.11.dev117+eb6b1f3 requests/2.11.1 CP...
ip,52.22.60.0
latitude,39.0481
longitude,-77.4728


In [6]:
def partition_file(args):
    #print(args)
    input_file, output_file = args
    import dask.dataframe as dd
    ddf = dd.read_parquet(input_file).repartition(partition_size='256MB')
    ddf.to_parquet(output_file, compression='SNAPPY')
    return input_file, output_file

In [7]:
import multiprocessing

In [8]:
p = multiprocessing.Pool(processes=6)

In [9]:
ret = p.imap_unordered(partition_file, parquet_files)
for idx, (in_file, out_file) in enumerate(ret):
    print(f'{idx+1}/{len(parquet_files)}')

1/1042
2/1042
3/1042
4/1042
5/1042
6/1042
7/1042
8/1042
9/1042
10/1042
11/1042
12/1042
13/1042
14/1042
15/1042
16/1042
17/1042
18/1042
19/1042
20/1042
21/1042
22/1042
23/1042
24/1042
25/1042
26/1042
27/1042
28/1042
29/1042
30/1042
31/1042
32/1042
33/1042
34/1042
35/1042
36/1042
37/1042
38/1042
39/1042
40/1042
41/1042
42/1042
43/1042
44/1042
45/1042
46/1042
47/1042
48/1042
49/1042
50/1042
51/1042
52/1042
53/1042
54/1042
55/1042
56/1042
57/1042
58/1042
59/1042
60/1042
61/1042
62/1042
63/1042
64/1042
65/1042
66/1042
67/1042
68/1042
69/1042
70/1042
71/1042
72/1042
73/1042
74/1042
75/1042
76/1042
77/1042
78/1042
79/1042
80/1042
81/1042
82/1042
83/1042
84/1042
85/1042
86/1042
87/1042
88/1042
89/1042
90/1042
91/1042
92/1042
93/1042
94/1042
95/1042
96/1042
97/1042
98/1042
99/1042
100/1042
101/1042
102/1042
103/1042
104/1042
105/1042
106/1042
107/1042
108/1042
109/1042
110/1042
111/1042
112/1042
113/1042
114/1042
115/1042
116/1042
117/1042
118/1042
119/1042
120/1042
121/1042
122/1042
123/1042
1

OSError: [Errno 28] No space left on device

Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-27:
Traceback (most recent call last):
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-6-37ebf54c9940>", line 5, in partition_file
    ddf = dd.read_parquet(input_file).repartition(partition_size='256MB')
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/dataframe/core.py", line 1125, in repartition
    return repartition_size(self, partition_size)
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/dataframe/core.py", line 5457, in repartition_size
    mem_usages = df.map_par

NameError: name 'ret' is not defined

In [63]:
list(ret)

KeyboardInterrupt: 

In [7]:
delayed_funcs = []
for idx, (in_file, out_file) in enumerate(parquet_files):
    print(f'{idx+1}/{len(parquet_files)}: {in_file} -> {out_file}')
    partition_file(in_file, out_file)

1/1042: /mnt/storage/anaconda-parsed-logs/2016/10/2016-10-28.parquet -> /mnt/storage/anaconda-parsed-logs-partitioned//2016/10/2016-10-28.parquet
2/1042: /mnt/storage/anaconda-parsed-logs/2016/10/2016-10-29.parquet -> /mnt/storage/anaconda-parsed-logs-partitioned//2016/10/2016-10-29.parquet


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-aa2a67c47572>", line 4, in <module>
    partition_file(input_file, output_file)
  File "<ipython-input-4-08efe50762a6>", line 4, in partition_file
    ddf.to_parquet(output_file)
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/dataframe/core.py", line 3629, in to_parquet
    return to_parquet(self, path, *args, **kwargs)
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/dataframe/io/parquet/core.py", line 483, in to_parquet
    out = out.compute()
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/base.py", line 175, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "/home/ericdill/miniconda/envs/dask/lib/python3.7/site-packages/dask/base.py", li

KeyboardInterrupt: 

In [1]:
dask.compute(delayed_funcs)

NameError: name 'dask' is not defined

In [ ]:
parquet_files

In [8]:
len(delayed_funcs)

34

In [9]:
dask.compute(delayed_funcs)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KilledWorker: ('partition_file-34163293-099c-445d-a5f3-c1f971a31c24', <Worker 'tcp://127.0.0.1:33070', memory: 0, processing: 20>)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [50]:
len(parquet_files)

1042

In [3]:
# cluster = LocalCUDACluster()

# client = distributed.Client(cluster)

In [4]:
year, month, day = '2019-09-01'.split('-')
fname = f'/mnt/storage/anaconda-parsed-logs/{year}/{month}/{year}-{month}-{day}.parquet'
partitioned_fname = f'/mnt/storage/parsed-logs-partitioned/{year}/{month}/{year}-{month}-{day}.parquet'
os.makedirs(os.path.dirname(partitioned_fname), exist_ok=True)

In [5]:
ddf = dd.read_parquet(fname)

In [8]:
ddf.head()

,bytes,city,continent,country,domain_name,full_user_agent,ip,label,latitude,longitude,...,pkg_python,pkg_version,postal_code,rayid,source,uri,user_agent,user_agent_version,user_platform,user_platform_version
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-09-01 00:00:00.072,452325,Ann Arbor,North America,United States,65.237.35.bc.googleusercontent.com,conda/4.5.4 requests/2.18.4 CPython/3.6.5 Linu...,35.237.65.0,,42.2734,-83.7133,...,,8.0,48104,50f2f6207888ba4c,cloudflare_anaconda,/conda-forge/linux-64/readline-8.0-hf8c457e_0....,Conda,4.5.4,Linux,4.9.0-9-amd64
2019-09-01 00:00:00.118,7165775,Grove City,North America,United States,zoominternet.net,conda/4.5.12 requests/2.21.0 CPython/3.7.1 Win...,24.144.174.0,NaN,41.1660,-80.0615,...,37,3.2.1,16127,50f2f620bb44a403,cloudflare_repo_anaconda,/pkgs/main/win-64/astropy-3.2.1-py37he774522_0...,Conda,4.5.12,Windows,10
2019-09-01 00:00:00.164,4213278,Bergen,Europe,Norway,bb.online.no,conda/4.3.34 requests/2.19.1 CPython/2.7.13 Li...,88.91.183.0,NaN,60.3266,5.3024,...,27,2.3.2,5152,50f2f6210824428f,cloudflare_repo_anaconda,/pkgs/free/linux-64/spyder-2.3.2-py27_0.tar.bz2,Conda,4.3.34,Linux,3.10.0-957.27.2.el7.x86_64
2019-09-01 00:00:00.173,6114,Barcelona,Europe,Spain,bsc.es,Mozilla,84.88.52.0,,41.3888,2.1590,...,,1.16.0,08034,50f2f6211ddf614f,cloudflare_anaconda,/bioconda/bioconductor-mirsynergy/1.16.0/downl...,Mozilla,,,
2019-09-01 00:00:00.224,6168,Barcelona,Europe,Spain,bsc.es,Mozilla,84.88.52.0,,41.3888,2.1590,...,,1.40.0,08034,50f2f6216a776161,cloudflare_anaconda,/bioconda/bioconductor-genoset/1.40.0/download...,Mozilla,,,


In [11]:
ddf = ddf.repartition(partition_size="256MB")

In [ ]:
ddf.to_parquet(partitioned_fname)

In [12]:
ddf.npartitions

10

In [10]:
ddf.to_parquet(partitioned_fname)

In [ ]:
import fastparquet